In [8]:
# 导入tushare
import tushare as ts
# 初始化pro接口
pro = ts.pro_api('3339f390298f8503dc5cbcca9fce9898f10bd3a5cb8ce9315803e5cd')

In [81]:
# 导入沪深300股票数据，提取股票代码
import pandas as pd
import numpy as np
stk_df = pd.read_csv('data/all_stock_data_20200101_20250922.csv')
all_stock_codes = stk_df['ts_code'].unique().tolist()

In [26]:
# 拉去数据的函数
def get_sw_industry_df(ts_code):
# 拉取数据
    df = pro.index_member_all(**{
        "l1_code": "",
        "is_new": "",
        "ts_code": ts_code,
        "src": "",
        "limit": "",
        "offset": ""
    }, fields=[
        "l1_code",
        "l1_name",
        "ts_code",
        "name",
        "in_date",
        "out_date",
        "is_new"
    ])
    return df

In [31]:
# 拉取所有股票的数据
from tqdm import tqdm  # 1. 导入tqdm
import time
stk_sw_industry_list = []
# 2. 将 all_stock_codes 用 tqdm() 包裹起来
for i in tqdm(all_stock_codes, desc="拉取各个股票对应的申万一级行业"):
    try:
        df_stk_ind = get_sw_industry_df(i)
        if df_stk_ind is not None and not df_stk_ind.empty:
            stk_sw_industry_list.append(df_stk_ind)
    except Exception as e:
        print(f"\n处理股票 {i} 时发生错误: {e}")  # 打印错误信息，\n确保不被进度条覆盖
    time.sleep(0.2)

stk_sw_industry_df = pd.concat(
    stk_sw_industry_list, ignore_index=True)

print("所有股票数据拉取并合并完成！")
stk_sw_industry_df.to_csv(
    'data/stk_sw_industry.csv', index=False)
print("数据已保存")

拉取各个股票对应的申万一级行业: 100%|██████████| 5634/5634 [42:25<00:00,  2.21it/s] 

所有股票数据拉取并合并完成！
数据已保存


In [40]:
stk_sw_industry_df

,l1_code,l1_name,ts_code,name,in_date,out_date,is_new
0,801150.SI,医药生物,688068.SH,热景生物,20190911,None,Y
1,801140.SI,轻工制造,002862.SZ,实丰文化,20170330,None,Y
2,801080.SI,电子,300408.SZ,三环集团,20140127,None,Y
3,801710.SI,建筑材料,600819.SH,耀皮玻璃,19940128,None,Y
4,801880.SI,汽车,300580.SZ,贝斯特,20161212,None,Y
...,...,...,...,...,...,...,...
5719,801080.SI,电子,600074.SH,退市保千(退市),20150401,None,Y
5720,801740.SI,国防军工,600677.SH,*ST航通(退市),20070702,None,Y
5721,801770.SI,通信,600485.SH,*ST信威(退市),20030804,None,Y
5722,801180.SI,房地产,600240.SH,退市华业(退市),20000628,None,Y


In [242]:
# 将个股行业信息添加到factor_df中, 对应起来
factor_df = pd.read_csv('result/factor_orthogonalized.csv')

In [243]:
factor_df = pd.merge(factor_df,stk_sw_industry_df[['ts_code','l1_code','l1_name']],on='ts_code')

In [244]:
# 将股票的收益率和流通市值添加到factor_df中
stk_df['trade_date']=pd.to_datetime(stk_df['trade_date'],format='%Y%m%d')
stk_df.sort_values(['ts_code','trade_date'],ascending=True,inplace=True)
stk_df = stk_df.reset_index(drop=True)
stk_df['ret'] = stk_df['close'].pct_change()

In [245]:
factor_df['trade_date'] = pd.to_datetime(factor_df['trade_date'])

In [246]:
barra_data = pd.merge(factor_df,stk_df[['ts_code','trade_date','ret','circ_mv']],on=['ts_code','trade_date'])

In [247]:
# 修改列名
barra_data.rename(columns={
    'trade_date':'date',
    'ts_code':'stocknames',
    'l1_code':'industry',
    'circ_mv':'capital',
    'SIZE':'size',
    'BETA':'beta',
    'MOMENTUM':'momentum',
    'VOLATILITY':'residual_volatility',
    'NLSIZE':'non_linear_size',
    'BP':'book_to_price_ratio',
    'LIQUIDITY':'liquidity',
    'EARNINGS':'earnings_yield',
    'GROWTH':'growth',
    'LEVERAGE':'leverage'
},inplace=True)

In [248]:
# 删去 "l1_name"
barra_data.drop(columns='l1_name',inplace=True)

In [249]:
factor_list = list(barra_data.columns[2:-3])

In [250]:
factor_list = ['size','beta','momentum','residual_volatility','non_linear_size',
               'book_to_price_ratio','liquidity','earnings_yield','growth','leverage']

In [251]:
barra_data

,stocknames,date,beta,size,momentum,residual_volatility,non_linear_size,book_to_price_ratio,liquidity,earnings_yield,growth,leverage,industry,ret,capital
0,000001.SZ,2020-01-02,NaN,17.304040,NaN,NaN,-15.409064,0.819001,NaN,0.085476,0.177005,92.230400,801780.SI,NaN,3.273750e+07
1,000001.SZ,2020-01-03,NaN,17.322249,NaN,NaN,-17.774646,0.804246,NaN,0.083933,0.177005,92.230400,801780.SI,0.018376,3.333908e+07
2,000001.SZ,2020-01-06,NaN,17.315826,NaN,NaN,-16.756745,0.809389,NaN,0.084474,0.177005,92.230400,801780.SI,-0.006403,3.312562e+07
3,000001.SZ,2020-01-07,NaN,17.320502,NaN,NaN,-16.518554,0.805607,NaN,5076.789733,0.177005,92.230400,801780.SI,0.004687,3.328087e+07
4,000001.SZ,2020-01-08,NaN,17.291514,NaN,NaN,-14.241812,0.829325,NaN,5226.107078,0.177005,92.230400,801780.SI,-0.028571,3.232998e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402045,688981.SH,2025-09-16,1.786026,18.290752,0.000859,0.047088,-43.446840,0.171739,1.805901,134.324440,0.025860,268.452347,801080.SI,0.010307,2.195120e+07
402046,688981.SH,2025-09-17,1.814614,18.357775,0.001058,0.038631,-52.496473,0.160606,1.858805,125.616637,0.025860,251.828937,801080.SI,0.069320,2.347286e+07
402047,688981.SH,2025-09-18,1.725423,18.397523,0.001019,0.062200,-61.447950,0.154576,1.894091,120.721548,0.025860,242.484091,801080.SI,0.040549,2.442466e+07
402048,688981.SH,2025-09-19,1.727354,18.390870,0.001002,0.061707,-60.553882,0.155608,1.931341,121.527419,0.025860,244.022519,801080.SI,-0.006631,2.426269e+07


In [252]:
barra_data = barra_data[['date','stocknames','capital','ret','industry']+factor_list]

In [253]:
barra_data['date'] = barra_data['date'].dt.strftime('%Y/%m/%d')

In [254]:
# ret 替换为 t+1 期的收益率
barra_data['ret'] = barra_data.groupby('stocknames')['ret'].shift(-1)

In [266]:
barra_data = barra_data[barra_data['date'] != '2025/09/22']

In [268]:
barra_data.to_csv('result/barra_data_csi.csv',index=False)

In [267]:
barra_data

,date,stocknames,capital,ret,industry,size,beta,momentum,residual_volatility,non_linear_size,book_to_price_ratio,liquidity,earnings_yield,growth,leverage
0,2020/01/02,000001.SZ,3.273750e+07,0.018376,801780.SI,17.304040,NaN,NaN,NaN,-15.409064,0.819001,NaN,0.085476,0.177005,92.230400
1,2020/01/03,000001.SZ,3.333908e+07,-0.006403,801780.SI,17.322249,NaN,NaN,NaN,-17.774646,0.804246,NaN,0.083933,0.177005,92.230400
2,2020/01/06,000001.SZ,3.312562e+07,0.004687,801780.SI,17.315826,NaN,NaN,NaN,-16.756745,0.809389,NaN,0.084474,0.177005,92.230400
3,2020/01/07,000001.SZ,3.328087e+07,-0.028571,801780.SI,17.320502,NaN,NaN,NaN,-16.518554,0.805607,NaN,5076.789733,0.177005,92.230400
4,2020/01/08,000001.SZ,3.232998e+07,0.007803,801780.SI,17.291514,NaN,NaN,NaN,-14.241812,0.829325,NaN,5226.107078,0.177005,92.230400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402044,2025/09/15,688981.SH,2.160551e+07,0.010307,801080.SI,18.279099,1.792031,0.000852,0.049337,-41.188569,0.173753,1.797395,135.898798,0.025860,271.457835
402045,2025/09/16,688981.SH,2.195120e+07,0.069320,801080.SI,18.290752,1.786026,0.000859,0.047088,-43.446840,0.171739,1.805901,134.324440,0.025860,268.452347
402046,2025/09/17,688981.SH,2.347286e+07,0.040549,801080.SI,18.357775,1.814614,0.001058,0.038631,-52.496473,0.160606,1.858805,125.616637,0.025860,251.828937
402047,2025/09/18,688981.SH,2.442466e+07,-0.006631,801080.SI,18.397523,1.725423,0.001019,0.062200,-61.447950,0.154576,1.894091,120.721548,0.025860,242.484091


In [ ]:
# 处理行业数据
industry_info = stk_sw_industry_df[['l1_code','l1_name','in_date']]
unique_industries_df = industry_info.drop_duplicates(subset=['l1_code', 'l1_name'])

In [124]:
unique_industries_df = unique_industries_df.reset_index(drop=True)

In [129]:
unique_industries_df['in_date'] = pd.to_datetime(unique_industries_df['in_date'],format='%Y%m%d')

In [131]:
unique_industries_df['in_date'] = unique_industries_df['in_date'].dt.strftime('%Y/%m/%d')

In [145]:
ind_info = unique_industries_df.copy()

In [146]:
ind_info.rename(columns={
    'l1_code':'code',
    'l1_name':'industry_names',
    'in_date':'start_date'
},inplace=True)

In [180]:
ind_factor_df = factor_df[['l1_code','l1_name']].drop_duplicates(subset=['l1_code', 'l1_name'])

In [183]:
ind_factor_df.reset_index(drop=True,inplace=True)

In [202]:
ind_factor_df = pd.merge(ind_factor_df, stk_sw_industry_df[['l1_code','l1_name','in_date']].drop_duplicates(subset=['l1_code', 'l1_name']), on=['l1_code', 'l1_name'], how='left')

In [203]:
ind_factor_df.rename(columns={
    'l1_code':'code',
    'l1_name':'industry_names',
    'in_date':'start_date'
},inplace=True)

In [204]:
ind_factor_df

,code,industry_names,start_date
0,801780.SI,银行,20160823
1,801180.SI,房地产,19981209
2,801770.SI,通信,20161201
3,801080.SI,电子,20140127
4,801890.SI,机械设备,20100812
5,801790.SI,非银金融,20220729
6,801960.SI,石油石化,20110126
7,801110.SI,家用电器,20230512
8,801880.SI,汽车,20161212
9,801050.SI,有色金属,20230710


In [206]:
ind_factor_df.to_csv('result/industry_info.csv',index=False)